In [ ]:
# Importar as bibliotecas necessárias

import json
import requests
import pandas as pd 
import time

In [ ]:
# Ler o arquivo, coloquei só 100 linhas como exemplo, por causa do limite de requisição da API, pois estou usando API FREE. E precisei fazer vários testes.

df = pd.read_csv("/home/cristina/Projetos/cnes-enriquecimento-dados/CNES/data/cnes_amostra.csv", encoding="latin1", low_memory=False,nrows=100)


Função cep_api_viacep, recebe um DataFrame contendo uma coluna chamada CO_CEP e faz requisições à API ViaCEP para obter os dados completos de endereço (como logradouro, bairro, cidade e estado) correspondentes a cada CEP.


• Utiliza a biblioteca requests para acessar a API.
• Percorre cada linha do DataFrame com itertuples().
• Para cada CEP:
   - Realiza a chamada à API https://viacep.com.br/ws/{cep}/json/.
   - Normaliza a resposta JSON em formato de DataFrame.
   - Concatena os resultados em um novo DataFrame chamado new_dataframe.
• Em caso de erro na requisição (ex: timeout ou CEP inválido), exibe uma mensagem e continua para o próximo CEP.
• Ao final, retorna um DataFrame com todos os dados de endereço obtidos.



In [ ]:

def cep_api_viacep(dataframe):
    new_dataframe = pd.DataFrame()

    for row in dataframe.itertuples():

        try:
            response = requests.get(f'https://viacep.com.br/ws/{row.CO_CEP}/json/',timeout=5)
            response.raise_for_status()
            response_df = pd.json_normalize(response.json())
            new_dataframe = pd.concat([new_dataframe, response_df], ignore_index=True)

        except requests.exceptions.RequestException as e:
            print(f"Erro com o CEP {row.CO_CEP}: {e}")
            continue
    
    return new_dataframe


In [ ]:
# Obtém os dados de endereço da API ViaCEP com base nos CEPs do DataFrame original
df_ceps = cep_api_viacep(df)

In [ ]:
# Remove o hífen dos CEPs retornados pela API para igualar ao formato da coluna "CO_CEP"
df_ceps["cep"] = df_ceps["cep"].str.replace("-", "")

In [68]:
# CO_CEP precisa estar como string com 8 dígitos (igual o 'cep' da API) para o merge funcionar
df['CO_CEP'] = df['CO_CEP'].astype(str).str.zfill(8)

In [ ]:
# Faz o merge dos dados originais com os dados da API, unindo pela coluna "CO_CEP" e "cep"
# Usa 'left join' para manter todos os registros do DataFrame original (df)

df_enriquecido = df.merge(df_ceps, left_on='CO_CEP', right_on='cep', how='left')

In [71]:
# Remover as linhas duplicadas e manter a primeira ocorrência
df_enriquecido = df_enriquecido.drop_duplicates(keep='first')

In [72]:
# Resetar o índice
df_enriquecido.reset_index(drop=True, inplace=True)

In [ ]:
# Deletar colunas repetidas e que não precisamos. 

df_enriquecido.drop(["Unnamed: 0","cep","ibge","gia","siafi","erro"], axis=1, inplace=True)

In [87]:
# Padronizar as colunas

df_enriquecido = df_enriquecido.rename(columns={"logradouro": "LOGRADOURO_ATUALIZADO",
                               "unidade": "UNIDADE",
                               "complemento":"COMPLEMENTO",
                               "bairro": "BAIRRO_ATUALIZADO",
                               "localidade": "CIDADE",
                               "uf": "UF",
                               "estado": "ESTADO",
                               "regiao": "REGIAO",
                               "ddd": "DDD"})

In [ ]:
# Padronizando os dados
colunas_api = ["LOGRADOURO_ATUALIZADO","UNIDADE","COMPLEMENTO","BAIRRO_ATUALIZADO","CIDADE","UF","ESTADO","REGIAO"]

for coluna_api in colunas_api:
    if coluna_api in df_enriquecido.columns:
        df_enriquecido[coluna_api] = df_enriquecido[coluna_api].str.upper()

In [ ]:
# Criar o arquivo com dados enriquecidos 
df_enriquecido.to_csv("/home/cristina/Projetos/cnes-enriquecimento-dados/CNES/data/cnes_dados_enriquecidos.csv", index=False)